In [ ]:
members = pickle.load(open("../Data_3/Data/members_MERGED.pkl", 'rb'))
# applications = pickle.load(open("../Data_3/Data/applications.pkl", 'rb'))
# appraisals_agriculture = pickle.load(open("../Data_3/Data/appraisals_agriculture.pkl", 'rb'))
# appraisals_business = pickle.load(open("../Data_3/Data/appraisals_business.pkl", 'rb'))
# appraisals_social = pickle.load(open("../Data_3/Data/appraisals_social.pkl", 'rb'))
# donations = pickle.load(open("../Data_3/Data/donations.pkl", 'rb'))
# loans = pickle.load(open("../Data_3/Data/loans.pkl", 'rb'))
# recoveries = pickle.load(open("../Data_3/Data/recoveries.pkl", 'rb'))
# schedules = pickle.load(open("../Data_3/Data/schedules.pkl", 'rb'))

In [240]:
# series = members['member id']

class Audit:
    import numpy as np
    import pandas as pd
    import pickle
    def __init__(self, checks_dict:dict):

        self.checks_dict = checks_dict
        self.results_lst = []
        self.numeric_funcs = []
        self.date_funcs = []
  #       self.empty_string_ = np.vectorize(self.empty_string)
		# self.only_numbers_ = np.vectorize(self.only_numbers)
		# self.extra_space_ = np.vectorize(self.extra_space)
		# self.trailing_space_ = np.vectorize(self.trailing_space)
		# self.leading_space_ = np.vectorize(self.leading_space)
		# self.less_then_n_unique_chars_ = np.vectorize(self.less_then_n_unique_chars)
		# self.case_problem_ = np.vectorize(self.case_problem)
		# self.only_one_unique_value_in_entire_column_ = np.vectorize(self.only_one_unique_value_in_entire_column)
		# self.nan_ = np.vectorize(self.nan)

        self.string_funcs = [
                            self.empty_string, 
                            self.only_numbers, 
                            self.extra_space, 
                            self.trailing_space, 
                            self.leading_space, 
                            self.less_then_n_unique_chars,
                            self.case_problem, 
                            self.only_one_unique_value_in_entire_column, 
                            self.nan
        ]

    def main(self):
        for data_name, checks in self.checks_dict.items():
            df = eval(data_name)
            if checks == "all":
                for col in df.columns:
                    if df[col].dtype == "O":
                        for func_name in self.string_funcs:
                            self.results_lst.append(func_name(series=df[col], data_name=data_name))
                    if df[col].dtype in ["float64", "int64"]:
                        for func_name in self.numeric_funcs:
                            self.results_lst.append(func_name(series=df[col], data_name=data_name))
                    if df[col].dtype == 'datetime64[ns]':
                        for func_name in self.date_funcs:
                            self.results_lst.append(func_name(series=df[col], data_name=data_name))
            # else:
                # for col
        pickle.dump(self.results_lst, open("results_lst.pkl", 'wb'))

    def empty_string(self, series, data_name):
        x = series.astype(str).str.strip().eq("")
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "empty_string",
            "Count" : x.sum(),
            "Perc" : round(x.mean()*100, 2)
        }

    def only_numbers(self, series, data_name):
        x = series.astype(str).str.split(".0", -1).str[0]
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "only_numbers",
            "Count" : x.str.isnumeric().sum(),
            "Perc" : round(x.str.isnumeric().mean()*100, 2),
            "Sample" : x.value_counts().drop(["nan", ''], errors="ignore").head().index.to_list()
        }

    def extra_space(self, series, data_name):
        x = series.astype(str).str.contains("  ")
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "extra_space",
            "Count" : x.sum(),
            "Perc" : round(x.mean()*100, 2),
            "Sample" : series[x].value_counts().head().index.to_list()
        }
    def trailing_space(self, series, data_name):
        x = series.astype(str).str.endswith(" ")
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "trailing_space",
            "Count" : x.sum(),
            "Perc" : round(x.mean()*100, 2),
            "Sample" : series[x].value_counts().head().index.to_list()
        }

    def leading_space(self, series, data_name):
        x = series.astype(str).str.startswith(" ")
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "leading_space",
            "Count" : x.sum(),
            "Perc" : round(x.mean()*100, 2),
            "Sample" : series[x].value_counts().head().index.to_list()
        }

    # same_char_more_than_n_times_in_sequence
    def less_then_n_unique_chars(self, series, data_name, n=5):
        x = series.astype(str).apply(lambda x:len(set(x)) < 5)
        return{
        	"Data" : data_name,
            "Column" : series.name,
            "Check" : f"less_then_{n}_unique_chars",
            "Count" : x.sum(),
            "Perc" : round(x.mean()*100, 2),
            "Sample" : series[x].value_counts().drop(["nan", ''], errors='ignore').head().index.to_list()
        }



    def only_one_unique_value_in_entire_column(self, series, data_name):

        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "only_one_unique_value_in_entire_column",
            "Value" : series.is_unique
        }
    def nan(self, series, data_name):
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "nan",
            "Count" : series.isna().sum(),
            "Perc" : round(series.isna().mean()*100, 2)
        }

    def case_problem(self, series, data_name):
        unique_vals = pd.Series(series.unique())
        x = (
            unique_vals
            .groupby(
                unique_vals
                .astype(str)
                .str
                .lower()
            )
            .size()
            .where(lambda x:x>2)
            .dropna()
            .index
            .to_list()
        )
        if not x:
            return {
	            "Data" : data_name,
                "Column" : series.name,
                'Check': 'case_problem',
                 'Count': 0,
                 'Perc': 0,
                 'Sample': []
            }
        x2 = series.isin(x)
        return {
            "Data" : data_name,
            "Column" : series.name,
            "Check" : "case_problem",
            "Count" : x2.sum(),
            "Perc" : round(x2.mean()*100, 2),
            "Sample" :     list(
            series[
                series
                .str
                .lower()
                .eq(
                    x
                    [0]
                )
            ].unique()
            )
        }

audit_obt = Audit({"members": "all"})
audit_obt.main()

from IPython.display import display
display(
	pd.DataFrame(audit_obt.results_lst)
    .where(lambda x:x.Count > 0)
    .dropna(how='all', axis=0)
    .dropna(how='all', axis=1)
    .fillna("-")
    .sort_values(["Data", "Column", "Check"])
)


,Data,Column,Check,Count,Perc,Sample
15,members,address,case_problem,10165.0,0.24,"[ Botta Park, Botta PARK, Botta PaRK]"
11,members,address,extra_space,170780.0,4.06,"[Orang Abad Jand Attock, P/o Box Nomal Tehsil..."
13,members,address,leading_space,66962.0,1.59,"[ CHAK NO 62 9 NANAKANA SB, ATARI SHAM SING B..."
14,members,address,less_then_5_unique_chars,108002.0,2.57,"[Tanda, Oghi, Bagh, Oggi, Thana]"
17,members,address,nan,85624.0,2.04,-
10,members,address,only_numbers,52041.0,1.24,"[0, Chak No , H#, 1, Chak No 1]"
12,members,address,trailing_space,470762.0,11.20,"[Pashat , Ghaila Kalan P/o Chakri , Khar Bajau..."
50,members,cnic,less_then_5_unique_chars,11513.0,0.27,"[33100-3313133-1, 33303-1003100-3, 13101-31300..."
53,members,cnic,nan,11466.0,0.27,-
46,members,cnic,only_numbers,4191203.0,99.71,"[35, 37, 36, 33, 34]"
